In [1]:
import polars as pl
import re
import emoji
import string


In [2]:
df=pl.read_csv("../dataset/data_preprocessed.csv")
df.head()

question,answer,context
str,str,str
"""من اصغر مساحة اليمن أم الأردن…","""الأردن""","""[' اليمن الجمهورية اليمنية …"
"""كم عدد Apple II CPUS&#39;s الت…","""العام""","""مع معالج Motorola 68030 الجديد…"
"""متى صدر فيلم المتحولون ؟""","""2007""",""" ترانسفورمرز أو المتحولون هو …"
"""هل بوروندي و بالاو لهما نفس جه…","""لا""","""["" جمهورية بوروندي é (فرنسي…"
"""من كان أكبر فنان كان لدى CBS؟""","""مايكل جاكسون""","""في 17 نوفمبر 1987 ، استحوذت SC…"


In [3]:
cols_to_process = ["question", "answer", "context"]

# Emoji regex pattern (vectorized)
emoji_pattern = emoji.get_emoji_regexp().pattern

# Export artifacts
GARBAGE_PATTERN = r"array\(|dtype=|\{'text':|\[|\]"

for col in cols_to_process:
    df = df.with_columns([
        pl.col(col)
        # Remove emojis first
        .str.replace_all(emoji_pattern, "", literal=False)
        # Remove export artifacts
        .str.replace_all(GARBAGE_PATTERN, "", literal=False)
        # Remove tatweel and diacritics
        .str.replace_all(r"[\u064B-\u0652\u0640]", "", literal=False)
        # Arabic letter normalization
        .str.replace_all(r"[إأآا]", "ا", literal=False)
        .str.replace_all(r"ى", "ي", literal=False)
        .str.replace_all(r"ؤ", "و", literal=False)
        .str.replace_all(r"ئ", "ي", literal=False)
        .str.replace_all(r"ة", "ه", literal=False)
        # Remove non-Arabic/non-numeric except punctuation
        .str.replace_all(r"[^\u0600-\u06FF0-9\s.,؟!]", " ", literal=False)
        # Normalize whitespace
        .str.replace_all(r"\s+", " ", literal=False)
        .str.strip_chars(" ")
        .alias(col)
    ])



In [4]:
garbage_pattern = r"array\(|dtype=|\{'text':|\[|\]"

df_clean = df.filter(
    ~pl.col("answer").str.contains(garbage_pattern) &
    pl.col("answer").is_not_null() &
    (pl.col("answer").str.strip_chars() != "")
)

print(f"Original size: {len(df)}")
print(f"Clean size: {len(df_clean)}")
print(f"Removed {len(df) - len(df_clean)} rows.")

print(df_clean.tail(1)[0, 1]) 

Original size: 194178
Clean size: 193509
Removed 669 rows.
1954


In [5]:
df_clean = df_clean.drop_nulls()

question,answer,context
u32,u32,u32
0,0,0


In [6]:
df_clean.tail()

question,answer,context
str,str,str
"""انتقلت حركه نهر الافق الجليدي …","""الوسط""","""درس اجاسيز حركه الانهار الجليد…"
"""هل الاردن يفوق موناكو مساحه ؟""","""نعم""","""الاردن المملكه الاردنيه الهاشم…"
"""من من هذين البلدين به متوسط عم…","""سان مارينو""","""سانت فينسنت والغرينادين بالانج…"
"""هل نسبه المياه بالارجنتين اقل …","""لا""","""جمهوريه الارجنتين 1 اسبانيه عل…"
"""متي اندلعت حرب الجزاير؟""","""1954""","""حرب الجزاير1، 2 3 هي حرب اندلع…"


In [7]:
df_clean.write_csv("../dataset/data_cleaned.csv")